In [2]:
import torch
import torch.nn as nn                 
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

from model.ViT import ViT, HopfieldViT
from model import *
# Define the Vision Transformer model

device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")

from tqdm import tqdm

import pandas as pd

import matplotlib.pyplot as plt 
# auto reload
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:

    
# Define the Vision Transformer model
class VisionTransformer(nn.Module):
    def __init__(self, num_classes, embed_dim, dim, num_heads, img_size, patch_size, in_channels=3):
        super(VisionTransformer, self).__init__()
        self.num_patches = (img_size // patch_size) ** 2
        self.patch_embedding = nn.Conv2d(in_channels, embed_dim, kernel_size=patch_size, stride=patch_size)
        self.positional_embedding = nn.Parameter(torch.randn(1, self.num_patches, embed_dim))
        self.hopfield = KHopfield(N=dim, n=embed_dim * self.num_patches)
        self.fc = nn.Linear(embed_dim * self.num_patches, num_classes)
        self.num_heads = num_heads

    def forward(self, x):
        x1 = self.patch_embedding(x)  # (batch_size, embed_dim, num_patches_h, num_patches_w)
        x2 = x1.permute(0, 2, 3, 1)  # (batch_size, num_patches_h, num_patches_w, embed_dim)
        x3 = x2.reshape(x2.size(0), -1, x2.size(-1))  # (batch_size, num_patches, embed_dim)
        
        x4 = x3 + self.positional_embedding  # Add positional embedding
        # combine second and third dimension
        x5 = x4.flatten(1, 2)
        x6 = self.hopfield(x5, self.num_heads)
        x7 = x6.mean(dim=2)  # Global average pooling
        x8 = self.fc(x7)
        return x8
    
    def to(self, device):
        super(VisionTransformer, self).to(device)
        self.hopfield = self.hopfield.to(device)
        return self
    
# Uses #num_heads k=1 hopfield networks, rather than k=num_heads-hopfield networks
class VisionTransformerV(nn.Module):
    def __init__(self, num_classes, embed_dim, dim, num_heads, img_size, patch_size, in_channels=3):
        super(VisionTransformerV, self).__init__()
        self.num_patches = (img_size // patch_size) ** 2
        self.patch_embedding = nn.Conv2d(in_channels, embed_dim, kernel_size=patch_size, stride=patch_size)
        self.positional_embedding = nn.Parameter(torch.randn(1, self.num_patches, embed_dim))

        self.hopfields = nn.ModuleList([KHopfield(N=dim, n=embed_dim * self.num_patches) for _ in range(num_heads)])
        self.fc = nn.Linear(embed_dim * self.num_patches, num_classes)
        self.num_heads = num_heads

    def forward(self, x):
        x1 = self.patch_embedding(x)  # (batch_size, embed_dim, num_patches_h, num_patches_w)
        x2 = x1.permute(0, 2, 3, 1)  # (batch_size, num_patches_h, num_patches_w, embed_dim)
        x3 = x2.reshape(x2.size(0), -1, x2.size(-1))  # (batch_size, num_patches, embed_dim)
        
        x4 = x3 + self.positional_embedding  # Add positional embedding
        # combine second and third dimension
        x5 = x4.flatten(1, 2)
        x6 = [self.hopfields[i](x5, 1) for i in range(self.num_heads)]
        # take average of all heads
        x6 = torch.stack(x6, dim=2).squeeze()
        x7 = x6.mean(dim=2)  # Global average pooling
        x8 = self.fc(x7)
        return x8
    


In [3]:
# # Define the Vision Transformer model
# class VisionTransformer(nn.Module):
#     def __init__(self, num_classes, embed_dim, num_heads, num_layers, img_size, patch_size):
#         super(VisionTransformer, self).__init__()
#         num_patches = (img_size // patch_size) ** 2
#         self.patch_embedding = nn.Conv2d(3, embed_dim, kernel_size=patch_size, stride=patch_size)
#         self.positional_embedding = nn.Parameter(torch.randn(1, num_patches + 1, embed_dim))
#         self.transformer = nn.Transformer(
#             d_model=embed_dim,
#             nhead=num_heads,
#             num_encoder_layers=num_layers,
#             dim_feedforward=2048,
#             dropout=0.1,
#         )
#         self.fc = nn.Linear(embed_dim, num_classes)

#     def forward(self, x):
#         x = self.patch_embedding(x)  # (batch_size, embed_dim, num_patches_h, num_patches_w)
#         x = x.permute(0, 2, 3, 1)  # (batch_size, num_patches_h, num_patches_w, embed_dim)
#         x = x.reshape(x.size(0), -1, x.size(-1))  # (batch_size, num_patches, embed_dim)
#         x = torch.cat([self.positional_embedding, x], dim=1)
#         x = self.transformer(x)
#         x = x.mean(dim=1)  # Global average pooling
#         x = self.fc(x)
#         return x


In [33]:
# Hyperparameters
batch_size = 256
num_epochs = 10
learning_rate = 1e-3
num_classes = 10
img_size = 32  # Assuming CIFAR-10 image size

#mnist

patch_size = 16  # Adjust this based on your preference

datast = 'mnist'
dataset = 'cifar'
num_heads = 4

if dataset == 'mnist':
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
    train_dataset = torchvision.datasets.MNIST(root='~/data', train=True, transform=transforms.ToTensor(), download=True)
    test_dataset = torchvision.datasets.MNIST(root='~/data', train=False, transform=transforms.ToTensor(), download=True)
    img_size = 28
    in_channels = 1
    num_classes = 10
else:
    train_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
                                    transforms.RandomCrop(32, padding=4), transforms.RandomHorizontalFlip()])

    val_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
    train_dataset = torchvision.datasets.CIFAR10(root='~/data', train=True, transform=train_transform, download=True)
    test_dataset = torchvision.datasets.CIFAR10(root='~/data', train=False, transform=val_transform, download=True)
    img_size = 32
    in_channels = 3
    num_classes = 10

# Data preprocessing
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


# Initialize the model and optimizer
model = VisionTransformer(
    num_classes = num_classes, 
    embed_dim = 256, 
    dim  = 1024,
    num_heads = num_heads, 
    img_size = img_size, 
    patch_size = patch_size, 
    in_channels = in_channels,
)
modelV= VisionTransformerV(
    num_classes = num_classes, 
    embed_dim = 256, 
    dim  = 1024,
    num_heads = num_heads, 
    img_size = img_size, 
    patch_size = patch_size, 
    in_channels = in_channels,
)

embedding_dim = 2196
num_heads = 8
num_layers = 1
patch_size = 16

modelS = SimpleViT(
    image_size = img_size,
    patch_size = patch_size,
    num_classes = num_classes,
    dim = 128,
    depth = 2,
    heads = 4,
    mlp_dim = 512
)

patch_size= 4
modelViT = ViT(
    image_size = img_size,
    patch_size = patch_size,
    num_classes = num_classes,
    dim = 128,
    depth = 2,
    heads = 4,
    mlp_dim = 512
)


patch_size= 4
modelViT = ViT(
    image_size = img_size,
    patch_size = patch_size,
    num_classes = num_classes,
    dim = 128,
    depth = 2,
    heads = 5,
    mlp_dim = 512
)

modelHViT = HopfieldViT(
    image_size = img_size,
    patch_size = patch_size,
    num_classes = num_classes,
    dim = 128,
    depth = 2,
    heads = 5,
    mlp_dim = 512
)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

model = model.to(device)
modelV = modelV.to(device)
print(device)






Files already downloaded and verified
Files already downloaded and verified
cuda:3


In [4]:
class TinyImagenetDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        #dataset =  load_dataset("zh-plus/tiny-imagenet", split='valid')
        self.imgs = []
        self.labels = []
        for i, d in enumerate(dataset):
            
            img = transforms.ToTensor()(d['image'])
            if img.shape[0] == 3 and img.shape[1] == 64 and img.shape[2] == 64:
                self.imgs.append(img)
                self.labels.append(d['label'])

        self.n = len(self.imgs)
        
    def __len__(self):
        return self.n
    
    def __getitem__(self, idx):
        return self.imgs[idx], self.labels[idx]
    
    def __repr__(self):
        return f"TinyImagenetDataset(n={self.n})"

In [6]:
from datasets import load_dataset
batch_size = 100
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
train_dataset = load_dataset("zh-plus/tiny-imagenet", split='train')
train_dataset = TinyImagenetDataset(train_dataset)
test_dataset = load_dataset("zh-plus/tiny-imagenet", split='valid')
test_dataset = TinyImagenetDataset(test_dataset)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
img_size = 64
in_channels = 3
num_classes = 200
modelViT = ViT(
    image_size = img_size,
    patch_size = patch_size,
    num_classes = num_classes,
    dim = 128,
    depth = 2,
    heads = 5,
    mlp_dim = 512
)
modelHViT = HopfieldViT(
    image_size = img_size,
    patch_size = patch_size,
    num_classes = num_classes,
    dim = 128,
    depth = 2,
    heads = 5,
    mlp_dim = 512
)

Using custom data configuration Maysee--tiny-imagenet-fd536421864cbb86
Reusing dataset parquet (/data/home/sjaffe/.cache/huggingface/datasets/zh-plus___parquet/Maysee--tiny-imagenet-fd536421864cbb86/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Using custom data configuration Maysee--tiny-imagenet-fd536421864cbb86
Reusing dataset parquet (/data/home/sjaffe/.cache/huggingface/datasets/zh-plus___parquet/Maysee--tiny-imagenet-fd536421864cbb86/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


NameError: name 'patch_size' is not defined

In [49]:
modelHViT = HopfieldViT(
    image_size = img_size,
    patch_size = patch_size,
    num_classes = num_classes,
    dim = 128,
    depth = 2,
    heads = 1,
    mlp_dim = 512
)

In [47]:
image, label = next(iter(train_loader))
image.shape

torch.Size([256, 3, 64, 64])

In [46]:
modelViT(image).shape

torch.Size([256, 200])

In [50]:
modelHViT(image).shape

torch.Size([256, 200])

In [7]:
x = torch.randn(256, 64)
print(x.shape)
result = LML3(N = 5, n_iter = 200, eps = 1e-3)(x)
print(result.shape)
print(torch.sum(result, dim=1))

torch.Size([256, 64])
torch.Size([256, 64])
tensor([5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000,
        5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000,
        5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000,
        5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000,
        5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000,
        5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000,
        5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000,
        5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000,
        5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000,
        5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000,
        5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000,
        5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.00

In [56]:
def ssm(x, k, beta):
    # x in b x n
    return LML2(N=k, n_iter = 200,eps = 1e-3)(beta * x)


In [57]:
param = torch.randn(256, 65, 65)
result = ssm(x, 5, 1e-3)

In [59]:
torch.sum(result, dim=-1)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [14]:
def validate(model, val_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in tqdm(val_loader):
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    return 100 * correct / total

In [17]:
# Training loop

model = modelViT.to(device)
modelV = modelHViT.to(device)
learning_rate = 1e-3
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):

    # show loss in tqdm
    model.train()
    pbar = tqdm(enumerate(train_loader), total=len(train_loader))
    for i, (images, labels) in pbar:
        images = images.to(device)
        labels = labels.to(device)

    
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        pbar.set_description(f'Epoch [{epoch + 1}/{num_epochs}] Loss: {loss.item():.4f}')

    # validate
    accuracy = validate(model, test_loader)

    print(f'Epoch [{epoch + 1}/{num_epochs}] Loss: {loss.item():.4f}, Val Accuracy: {accuracy:.4f}')
    

print('Training finished!')

  0%|          | 0/196 [00:00<?, ?it/s]

100%|██████████| 40/40 [00:02<00:00, 13.53it/s]


Epoch [1/10] Loss: 1.5576, Val Accuracy: 41.8200


100%|██████████| 40/40 [00:02<00:00, 13.42it/s]


Epoch [2/10] Loss: 1.4186, Val Accuracy: 47.3200


100%|██████████| 40/40 [00:02<00:00, 13.43it/s]


Epoch [3/10] Loss: 1.2483, Val Accuracy: 52.0500


Epoch [4/10] Loss: 1.3388:  38%|███▊      | 74/196 [00:09<00:16,  7.44it/s]


KeyboardInterrupt: 

In [8]:
X = torch.nn.Parameter(torch.randn(59, 16))

hopfield = KHopfield(N=100, n=16 )

# optimize the hopfield network
optimizer = optim.Adam(hopfield.parameters(), lr=learning_rate)

for i in range(10):
    print(i)
    Y = hopfield(X, 4)
    loss = torch.norm(Y)
    loss.backward()
    print(loss)
    optimizer.step()

0
tensor(0.8274, grad_fn=<CopyBackwards>)
1
tensor(0.7943, grad_fn=<CopyBackwards>)
2
tensor(0.7643, grad_fn=<CopyBackwards>)
3
tensor(0.7368, grad_fn=<CopyBackwards>)
4
tensor(0.7115, grad_fn=<CopyBackwards>)
5
tensor(0.6887, grad_fn=<CopyBackwards>)
6
tensor(0.6683, grad_fn=<CopyBackwards>)
7
tensor(0.6504, grad_fn=<CopyBackwards>)
8
tensor(0.6348, grad_fn=<CopyBackwards>)
9
tensor(0.6215, grad_fn=<CopyBackwards>)


In [33]:
# Initialize the model and optimizer
num_heads = 3


model = VisionTransformer(
    num_classes = num_classes, 
    embed_dim = 1024, 
    num_heads = num_heads, 
    img_size = img_size, 
    patch_size = patch_size, 
    in_channels = in_channels,
)
model = SimpleViT(
    image_size = img_size,
    patch_size = patch_size,
    num_classes = num_classes,
    dim = 1024,
    depth = 1,
    heads = 4,
    mlp_dim = 2048
)

transformer 1


In [32]:
def get_model_and_data(
        data = 'cifar10',
        model_name = 'vit_hopfield',
        batch_size = 256,
        heads = 4):
    if data == 'mnist':
        transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
        train_dataset = torchvision.datasets.MNIST(root='~/data', train=True, transform=transforms.ToTensor(), download=True)
        test_dataset = torchvision.datasets.MNIST(root='~/data', train=False, transform=transforms.ToTensor(), download=True)
        img_size = 28
        in_channels = 1
        num_classes = 10

    elif data  == 'cifar10':
        transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465),(0.2023, 0.1994, 0.2010))])

        train_dataset = torchvision.datasets.CIFAR10(root='~/data', train=True, transform=transform, download=True)
        test_dataset = torchvision.datasets.CIFAR10(root='~/data', train=False, transform=transform, download=True)
        img_size = 32
        in_channels = 3
        num_classes = 10
    else:
        raise Exception('data not found')
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



    patch_size= 4
    if model_name == 'vit':
        model = ViT(
            image_size = img_size,
            patch_size = patch_size,
            num_classes = num_classes,
            dim = 128,
            depth = 2,
            heads = heads,
            mlp_dim = 512,
            channels = in_channels,
        )
    elif model_name == 'vit_hopfield':
        model = HopfieldViT(
            image_size = img_size,
            patch_size = patch_size,
            num_classes = num_classes,
            dim = 128,
            depth = 2,
            heads = heads,
            mlp_dim = 512,
            channels = in_channels,
        )
    return model, train_loader, test_loader

In [20]:
def run_experiment(train_loader, test_loader, model, num_epochs=30, k=1):
    # Initialize the model and optimizer
    model = model.to(device)
    learning_rate = 1e-3
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    val_accuracy = []
    # Training loop
    for epoch in range(num_epochs):

        # show loss in tqdm
        model.train()
        pbar = tqdm(enumerate(train_loader), total=len(train_loader))
        for i, (images, labels) in pbar:
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            pbar.set_description(f'K: {k}, Epoch [{epoch + 1}/{num_epochs}] Loss: {loss.item():.4f}')

        # validate
        accuracy = validate(model, test_loader)

        print(f'Epoch [{epoch + 1}/{num_epochs}] Loss: {loss.item():.4f}, Val Accuracy: {accuracy:.4f}')
        val_accuracy.append(accuracy)

    print('Training finished!')

    return val_accuracy

In [35]:
model, train_loader, test_loader = get_model_and_data(data = 'cifar10', model_name = 'vit', batch_size = 256, heads = 1)

Files already downloaded and verified
Files already downloaded and verified


In [36]:
val_accuracy  = run_experiment(train_loader, test_loader, model)

100%|██████████| 40/40 [00:02<00:00, 14.26it/s]


Epoch [1/30] Loss: 1.6778, Val Accuracy: 39.5100


K: 1, Epoch [2/30] Loss: 1.5923:  16%|█▌        | 31/196 [00:02<00:14, 11.07it/s]


KeyboardInterrupt: 

In [7]:
ks = [1, 4, 8, 16]
df = pd.DataFrame(columns = ['k', 'accuracy', 'model', 'epoch'])
data = 'cifar10'
model_name = 'hopfield'
for k in ks:
    model, train_loader, test_loader = get_model_and_data(data = data, model = model_name, batch_size = 256, heads = k, dim=100)
    val_accuracy  = run_experiment(train_loader, test_loader, model, num_epochs=1, k =k )
    # add every val accuracy to dataframe
    for i, acc in enumerate(val_accuracy):
        df = df.append({'k': k, 'accuracy': acc, 'model': 'hopfield', 'epoch': i}, ignore_index=True)
    
    # save dataframe
    df.to_csv(f'./results/{data}_{model_name}_heads.csv')

Files already downloaded and verified
Files already downloaded and verified


K: 1, Epoch [1/1] Loss: 1.8465:  74%|███████▍  | 145/196 [00:12<00:05,  8.73it/s]

LML Warning: Did not converge.


K: 1, Epoch [1/1] Loss: 1.8947:  80%|████████  | 157/196 [00:14<00:04,  8.61it/s]

LML Warning: Did not converge.


...

LML Warning: Did not converge.
LML Warning: Did not converge.


 92%|█████████▎| 37/40 [00:05<00:00,  5.49it/s]

LML Warning: Did not converge.
LML Warning: Did not converge.


100%|██████████| 40/40 [00:06<00:00,  6.55it/s]

LML Warning: Did not converge.
LML Warning: Did not converge.
Epoch [1/1] Loss: 2.0308, Val Accuracy: 34.2100
Training finished!


In [8]:
# get df rows
import pandas as pd
data = 'cifar10'
model_name = 'hopfield'
df = pd.read_csv(f'./results/{data}_{model_name}_heads.csv')

In [9]:
df

,Unnamed: 0,k,accuracy,model,epoch
0,0,1,29.75,hopfield,0
1,1,2,34.21,hopfield,0


In [4]:
print(f'./results/{data}_{model}_heads.csv')

./results/cifar10_hopfield_heads.csv


In [11]:
data = 'mnist'
model = 'hopfield'
df = pd.read_csv(f'./results/{data}_{model}_heads.csv')

In [12]:
df

,Unnamed: 0,k,accuracy,model,epoch
0,0,4,15.41,hopfield,0
1,1,4,17.31,hopfield,1
2,2,4,17.99,hopfield,2
3,3,4,18.91,hopfield,3
4,4,4,17.94,hopfield,4
...,...,...,...,...,...
505,505,16,49.78,hopfield,25
506,506,16,50.14,hopfield,26
507,507,16,51.12,hopfield,27
508,508,16,52.14,hopfield,28


In [4]:
ks= [1, 4, 8, 16]

data = 'cifar100'
data = 'tinyimagenet'
n=30
model = 'vit'
df = pd.read_csv(f'./results/{data}_{model}_heads.csv')
# color scheme by k
colors = ['red', 'blue', 'green', 'orange']
for i, k in enumerate(ks):
    df_k = df[df['k'] == k]
    # take only last 30 rows
    df_k = df_k.iloc[-n:]

    plt.plot(df_k['epoch'], df_k['accuracy'], label=f'k={k}', color=colors[i] )
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Test Accuracy')
plt.title(f'Vision Transformer')

plt.show()
model = 'vit_hopfield'
df = pd.read_csv(f'./results/{data}_{model}_heads.csv')
for i, k in enumerate(ks):
    df_k = df[df['k'] == k]
    # take only last 30 rows
    df_k = df_k.iloc[-n:]
    # dash lines
    plt.plot(df_k['epoch'], df_k['accuracy'], label=f'k={k}', linestyle='--', color=colors[i] )
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Test Accuracy')

plt.title('KHopfield Transformer')
plt.show()

# model = 'vit'
# df = pd.read_csv(f'./results/{data}_{model}_heads.csv')
# for i, k in enumerate(ks):
#     df_k = df[df['k'] == k]
#     # take only last 30 rows
#     df_k = df_k.iloc[-30:]
#     # dash lines
#     plt.plot(df_k['epoch'], df_k['accuracy'], label=f'k={k}', linestyle=':', color=colors[i] )




In [14]:
k= 4
df_k = df[df['k'] == k]

In [21]:
# number of rows in df_k
k =4
df_k = df[df['k'] == k]
df_k.shape[0]

150

In [6]:
data = 'cifar10'
model = 'vit_hopfield'
k = 1
df = pd.read_csv(f'./results/{data}_{model}_heads.csv')
df_k = df[df['k'] == k]

In [7]:

plt.plot( df_k['accuracy'].to_numpy(), label=f'k={k}')